# 3. Train model (Pytorch)

In [1]:
import os
import math
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from metrics import evaluate_all
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
ROOT = Path(os.path.abspath('')).resolve().parents[0]
DATA = os.path.join(ROOT, "data")
INTERIM_DATA = os.path.join(DATA, "interim")
RAW_DATA = os.path.join(DATA, "raw")
MODELS = os.path.join(DATA, "models")
MOVIELENS_PATH = os.path.join(RAW_DATA, "ml-1m")

In [3]:
users_df = pd.read_parquet(os.path.join(INTERIM_DATA, 'users.parquet.gzip'))
ratings_df = pd.read_parquet(os.path.join(INTERIM_DATA, 'ratings.parquet.gzip'))
movies_df = pd.read_parquet(os.path.join(INTERIM_DATA, 'movies.parquet.gzip'))

In [4]:
def temporal_split_per_user(
    ratings: pd.DataFrame,
    n_val: int = 1,
    n_test: int = 1,
    min_train: int = 5,
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Per-user time-based split:
    - sort each user's interactions by timestamp
    - last n_test -> test
    - previous n_val -> val
    - rest -> train
    Users with too few interactions are kept in train only.
    """
    r = ratings.sort_values(["user_id", "timestamp"]).copy()

    r["rank"] = r.groupby("user_id").cumcount() + 1
    r["user_cnt"] = r.groupby("user_id")["movie_id"].transform("size")

    eligible = r["user_cnt"] >= (min_train + n_val + n_test)

    test_mask = eligible & (r["rank"] > r["user_cnt"] - n_test)
    val_mask  = eligible & (r["rank"] > r["user_cnt"] - (n_test + n_val)) & ~test_mask
    train_mask = ~test_mask & ~val_mask

    train = r.loc[train_mask].drop(columns=["rank", "user_cnt"])
    val   = r.loc[val_mask].drop(columns=["rank", "user_cnt"])
    test  = r.loc[test_mask].drop(columns=["rank", "user_cnt"])

    return train, val, test


train_df, val_df, test_df = temporal_split_per_user(ratings_df, n_val=1, n_test=5, min_train=5)

print(train_df.shape, val_df.shape, test_df.shape)
print("Users in val:", val_df["user_id"].nunique(), "Users in test:", test_df["user_id"].nunique())

(963969, 4) (6040, 4) (30200, 4)
Users in val: 6040 Users in test: 6040


## Sanity checks

### No intersections by lines

In [5]:
assert set(map(tuple, train_df[["user_id","movie_id","timestamp"]].values)).isdisjoint(
       set(map(tuple, test_df[["user_id","movie_id","timestamp"]].values)))

### Users from val/test have data in train

In [6]:
users_train = set(train_df["user_id"].unique())
assert set(val_df["user_id"].unique()).issubset(users_train)
assert set(test_df["user_id"].unique()).issubset(users_train)

### Order by time within a user: max(train) <= min(test) (for eligible users)

In [7]:
tmp = ratings_df.sort_values(["user_id","timestamp"])
tmp

,user_id,movie_id,rating,timestamp
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


## Baseline 1 - Popularity

To verify the correctness and usefulness of more advanced recommendation models, it is crucial to establish a simple yet strong baseline.
The popularity-based recommender serves as such a reference point.

This baseline recommends the same set of items to all users, selecting the most popular movies according to the number of positive interactions in the training data (ratings ≥ 4). No personalization is involved.

The popularity baseline provides a lower bound on model performance:
* it represents the best result achievable without any personalization,
* it reflects how much of the recommendation quality can be explained solely by globally popular items,
* it helps detect implementation errors in more complex models — if a personalized model fails to outperform this baseline, it is likely misconfigured or ineffective.

From a practical perspective, popularity-based recommendations are often used in real systems as:
* a fallback strategy for cold-start users,
* a simple default solution,
* or a component of a larger ensemble.

### Implementation details

The model is implemented by:
1. Filtering the training data to keep only positive interactions (rating ≥ 4).
2. Counting the number of such interactions for each movie.
3. Ranking movies by their popularity score.
4. Returning the top-K most popular movies, excluding items already seen by the user (when applicable).

Formally, for each item i, its popularity score is defined as:

$$
\huge popularity(i) = \sum_{u}\mathbb{1}[r_{u,i} \ge 4]
$$

The recommendation function returns the top-K items with the highest popularity scores.

This baseline establishes a clear reference point:
any personalized model (e.g. matrix factorization, ALS, neural models) is expected to surpass this score in terms of ranking metrics such as `Recall@K` and `NDCG@K`.

In [8]:
train_df["interaction"] = (train_df["rating"] >= 4).astype(int)
val_df["interaction"] = (val_df["rating"] >= 4).astype(int)
test_df["interaction"] = (test_df["rating"] >= 4).astype(int)

In [9]:
train_pos = train_df[train_df["interaction"] == 1]

In [10]:
item_popularity = (
    train_pos
    .groupby("movie_id")
    .size()
    .sort_values(ascending=False)
)

item_popularity.head()

movie_id
2858    2766
260     2557
1196    2461
1198    2213
2028    2194
dtype: int64

## Baseline 2 - neural network model

Explicit ratings are converted into implicit positive interactions.

Only positive feedback (rating >= 4) is retained. 

The absence of an interaction is treated as unknown rather than negative feedback:

In [11]:
train_tmp = train_df.copy()
train_tmp["interaction"] = (train_tmp["rating"] >= 4).astype(np.int8)
train_pos = train_tmp[train_tmp["interaction"] == 1][["user_id", "movie_id"]].copy()

### Index mapping and factorization

User and item identifiers are mapped to consecutive integer indices to enable efficient matrix operations.

Bidirectional mappings (user2idx, idx2item, etc.) are stored to convert between internal indices and original identifiers during inference:

In [12]:
u_codes, u_uniques = pd.factorize(train_pos["user_id"], sort=True)
i_codes, i_uniques = pd.factorize(train_pos["movie_id"], sort=True)

### Sparse interaction matrix

A sparse user–item interaction matrix $\large X_{ui}$ is constructed.

Here, $\large X_{ui} = 1$ indicates a positive interaction, and zero indicates no observed interaction.

In [13]:
train_pos["u_idx"] = u_codes.astype(np.int32)
train_pos["i_idx"] = i_codes.astype(np.int32)

user2idx = pd.Series(np.arange(len(u_uniques)), index=u_uniques).to_dict()
idx2user = pd.Series(u_uniques).to_dict()

idx2item = pd.Series(i_uniques).to_dict()          # i_idx -> movie_id
item2idx = pd.Series(np.arange(len(i_uniques)), index=i_uniques).to_dict()

X_ui = csr_matrix(
    (np.ones(len(train_pos), dtype=np.float32),
     (train_pos["u_idx"].to_numpy(), train_pos["i_idx"].to_numpy())),
    shape=(len(u_uniques), len(i_uniques)),
)

#### Sanity checks

In [14]:
print("X_ui shape:", X_ui.shape)
print("max i_idx:", train_pos["i_idx"].max(), "idx2item max key:", max(idx2item.keys()))

X_ui shape: (6038, 3525)
max i_idx: 3524 idx2item max key: 3524


## Neural Matrix Factorization (NeuralMF): Model Explanation

### What this model does

`NeuralMF` is a **neural collaborative filtering model** designed to predict how relevant an item is for a given user.  
For each `(user, item)` pair, the model outputs a **scalar score (logit)** that represents the strength of preference.  
Higher scores indicate a higher likelihood that the user will interact positively with the item.

The model is typically trained on **implicit feedback** (e.g. clicks or ratings above a threshold) using a loss such as  
`BCEWithLogitsLoss`, making it suitable for **top-K recommendation** tasks.

---

### High-level idea

The model combines two core ideas:

1. **Matrix Factorization**  
   Users and items are represented as dense latent vectors (embeddings).

2. **Neural interaction modeling**  
   Instead of a simple dot product, a neural network learns a **non-linear interaction function** between user and item embeddings.

This makes the model more expressive than classical matrix factorization while remaining computationally efficient.

---

### Architecture breakdown

#### 1. User and item embeddings

Each user and each item is mapped to a dense vector:

- `user_emb`: user latent representation  
- `item_emb`: item latent representation  

The embedding dimension (`emb_dim`) controls the capacity of the latent space.

Small random initialization (`std = 0.01`) helps:
- stabilize early training
- avoid overly confident initial predictions

This component is directly analogous to latent factors in classical MF models.

---

#### 2. Embedding concatenation

User and item embeddings are **concatenated**:

- preserves full information from both embeddings
- allows the model to learn cross-feature interactions
- produces a vector of size `2 × emb_dim`

Unlike dot-product MF, no assumption of linear interaction is made at this stage.

---

#### 3. Multi-Layer Perceptron (MLP)

The concatenated embeddings are passed through a small MLP:

- **Linear layer**: projects embeddings into an interaction space
- **ReLU activation**: introduces non-linearity
- **Dropout**: regularization to reduce overfitting
- **Final linear layer**: outputs a single scalar score

This network learns how user and item features interact in a non-linear way.

---

#### 4. Model output

The model outputs a **logit** (raw score) for each `(user, item)` pair:

- can be passed through `sigmoid` for probability estimation
- can be used directly for ranking items
- supports binary classification and ranking losses

---

### Why this architecture was chosen

#### Expressiveness over classical MF
- Classical MF uses a dot product (linear interaction)
- NeuralMF learns complex, non-linear interactions
- Better captures real-world user preferences

#### Simplicity and scalability
- Lightweight architecture
- Efficient to train and infer
- Scales to large numbers of users and items

#### Compatibility with implicit feedback
- Works naturally with negative sampling
- Suitable for ranking-based recommendation
- Produces scores ideal for top-K retrieval

#### Strong baseline neural recommender
- Conceptually related to GMF / NeuMF
- Easy to extend (deeper MLP, additional features)
- Practical for both research and production

---

### When this model works best

- Medium to large implicit feedback datasets
- Sparse user–item interaction matrices
- Scenarios where interactions are non-linear
- As a baseline before introducing more complex architectures

---

### Limitations

- Scores all items per user (O(n_items)), requiring batching or candidate generation
- Does not model temporal or sequential behavior
- Requires storing user/item mappings and interaction history

---

### Model evaluations

In [15]:
def ndcg_at_k(recommended, relevant, k=10):
    rel = set(relevant)
    dcg = 0.0
    for i, item in enumerate(recommended[:k], start=1):
        if item in rel:
            dcg += 1.0 / math.log2(i + 1)
    ideal_hits = min(len(rel), k)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, ideal_hits + 1))
    return 0.0 if idcg == 0 else dcg / idcg

def evaluate_model(recommend_fn, model, test_df, k=10, threshold=4):
    tmp = test_df.copy()
    tmp["interaction"] = (tmp["rating"] >= threshold).astype(np.int8)
    test_pos = tmp[tmp["interaction"] == 1]

    recalls, ndcgs = [], []
    for user_id, g in test_pos.groupby("user_id"):
        relevant = g["movie_id"].tolist()
        recs = recommend_fn(model, user_id, k=k)
        if not recs:
            continue

        recalls.append(len(set(recs) & set(relevant)) / len(relevant))
        ndcgs.append(ndcg_at_k(recs, relevant, k=k))

    return float(np.mean(recalls)), float(np.mean(ndcgs))

In [16]:
def build_train_pairs_with_negatives(train_pos: pd.DataFrame, X_ui, n_items: int, n_neg: int = 4, seed: int = 42):
    """
    train_pos: DataFrame [u_idx, i_idx] positive interactions
    X_ui: csr_matrix (n_users, n_items) positives as 1
    returns arrays: users, items, labels
    """
    rng = np.random.default_rng(seed)

    pos_u = train_pos["u_idx"].to_numpy()
    pos_i = train_pos["i_idx"].to_numpy()
    n_pos = len(pos_u)

    users = [pos_u]
    items = [pos_i]
    labels = [np.ones(n_pos, dtype=np.float32)]

    neg_users = np.repeat(pos_u, n_neg)
    neg_items = np.empty(n_pos * n_neg, dtype=np.int64)

    idx = 0
    for u in pos_u:
        liked = set(X_ui.getrow(u).indices)
        for _ in range(n_neg):
            j = int(rng.integers(0, n_items))
            while j in liked:
                j = int(rng.integers(0, n_items))
            neg_items[idx] = j
            idx += 1

    users.append(neg_users)
    items.append(neg_items)
    labels.append(np.zeros(len(neg_users), dtype=np.float32))

    users = np.concatenate(users)
    items = np.concatenate(items)
    labels = np.concatenate(labels)

    perm = rng.permutation(len(users))
    return users[perm], items[perm], labels[perm]

### Dataset

In [17]:
class PairDataset(Dataset):
    def __init__(self, users, items, labels):
        self.users = torch.tensor(users, dtype=torch.long)
        self.items = torch.tensor(items, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

### Model definition

In [18]:
class NeuralMF(nn.Module):
    def __init__(self, n_users: int, n_items: int, emb_dim: int = 64, hidden_dim: int = 128, dropout: float = 0.1):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(emb_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
        )

        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)

    def forward(self, u, i):
        ue = self.user_emb(u)
        ie = self.item_emb(i)
        x = torch.cat([ue, ie], dim=1)
        logit = self.mlp(x).squeeze(1)
        return logit

### Recommendation function

In [19]:
@torch.no_grad()
def recommend_nmf(model, user_id: int, k: int = 10, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    if user_id not in user2idx:
        return []

    uidx = user2idx[user_id]
    model.eval()
    model.to(device)

    n_items = X_ui.shape[1]

    item_idx = torch.arange(n_items, dtype=torch.long, device=device)
    user_idx = torch.full((n_items,), uidx, dtype=torch.long, device=device)

    bs = 20000
    scores = []
    for start in range(0, n_items, bs):
        end = min(start + bs, n_items)
        scores.append(model(user_idx[start:end], item_idx[start:end]).detach().cpu().numpy())
    scores = np.concatenate(scores)

    # exclude seen positives from train
    seen = set(X_ui.getrow(uidx).indices)
    if seen:
        scores[list(seen)] = -np.inf

    top = np.argpartition(-scores, k)[:k]
    top = top[np.argsort(-scores[top])]
    return [idx2item[int(i)] for i in top]

### Training function

In [20]:
def train_neural_mf(
    model: nn.Module,
    train_pos: pd.DataFrame,
    X_ui,
    n_items: int,
    test_df,
    n_neg: int = 4,
    epochs: int = 3,
    batch_size: int = 4096,
    lr: float = 1e-3,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
    weight_decay=1e-6,
):
    model = model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(1, epochs + 1):
        users, items, labels = build_train_pairs_with_negatives(
            train_pos=train_pos,
            X_ui=X_ui,
            n_items=n_items,
            n_neg=n_neg,
            seed=42 + epoch,
        )
        ds = PairDataset(users, items, labels)
        dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)

        model.train()
        total_loss = 0.0
        for u, i, y in dl:
            u, i, y = u.to(device), i.to(device), y.to(device)

            opt.zero_grad()
            logit = model(u, i)
            loss = criterion(logit, y)
            loss.backward()
            opt.step()

            total_loss += loss.item() * len(u)

        avg_loss = total_loss / len(ds)
        if epoch % 2 == 0:
            r, n = evaluate_model(
                recommend_fn=lambda m, uid, k=10: recommend_nmf(m, uid, k=k, device=device),
                model=model,
                test_df=test_df,
                k=10,
            )
            print(f"  Eval Recall@10={r:.4f} NDCG@10={n:.4f}")

    return model

### Train

In [21]:
n_users, n_items = X_ui.shape
nmf = NeuralMF(n_users=n_users, n_items=n_items, emb_dim=64, hidden_dim=128, dropout=0.1)

nmf = train_neural_mf(
    model=nmf,
    train_pos=train_pos,
    X_ui=X_ui,
    n_items=n_items,
    test_df=test_df,
    epochs=20,
    batch_size=4096,
    lr=1e-3,
)

  Eval Recall@10=0.0451 NDCG@10=0.0314
  Eval Recall@10=0.0477 NDCG@10=0.0327
  Eval Recall@10=0.0531 NDCG@10=0.0374
  Eval Recall@10=0.0565 NDCG@10=0.0400
  Eval Recall@10=0.0574 NDCG@10=0.0410
  Eval Recall@10=0.0654 NDCG@10=0.0465
  Eval Recall@10=0.0670 NDCG@10=0.0471
  Eval Recall@10=0.0644 NDCG@10=0.0464
  Eval Recall@10=0.0666 NDCG@10=0.0472
  Eval Recall@10=0.0673 NDCG@10=0.0469


### Validate

In [22]:
recall10_bpr, ndcg10_bpr = evaluate_model(
    recommend_fn=lambda m, uid, k=10: recommend_nmf(m, uid, k=k),
    model=nmf,
    test_df=test_df,
    k=10,
)
recall10_bpr, ndcg10_bpr

(0.06734798007083258, 0.04691890479314088)

### Metrics

In [23]:
recommend_nmf_fn = lambda uid, k: recommend_nmf(nmf, uid, k=k)
metrics_nmf = evaluate_all(
    recommend_nmf_fn,
    test_df,
    users_for_coverage=test_df["user_id"].unique()[:1000],
    all_items=movies_df["movie_id"].unique(),
    item_popularity=item_popularity.to_dict(),
    k=10,
)
metrics_nmf

{'recall@10': 0.06734798007083258,
 'ndcg@10': 0.04691890479314088,
 'mrr@10': 0.06718333290456467,
 'hitrate@10': 0.19034755987754368,
 'n_users_eval': 5553,
 'coverage@10': 0.1702292042235385,
 'avg_popularity@10': 1332.9576}

### Observations

Based on the evaluation metrics computed for the neural recommendation model, the following conclusions can be drawn.

---

#### Overall Recommendation Quality

- **Recall@10 ≈ 0.069**  
  The model retrieves at least one relevant item for about **6.9%** of users within the top-10 recommendations.  
  This indicates **low overall recall**, suggesting that the model struggles to consistently surface relevant items for most users.

- **NDCG@10 ≈ 0.048**  
  The relatively low NDCG score implies that **relevant items are not ranked highly** when they do appear.  
  Even when the model finds relevant items, their positions in the recommendation list are often suboptimal.

- **MRR@10 ≈ 0.069**  
  The Mean Reciprocal Rank confirms that the **first relevant item typically appears late** in the ranking, reinforcing the observation of weak ranking quality.

---

#### Hit-Based Evaluation

- **HitRate@10 ≈ 0.193**  
  Roughly **19% of users** receive at least one relevant recommendation in the top-10.  
  This suggests that while the model occasionally succeeds, its performance is **inconsistent across users**.

---

#### Coverage and Diversity

- **Coverage@10 ≈ 0.182**  
  About **18% of the item catalog** appears in recommendations, indicating **limited item diversity**.  
  The model concentrates recommendations on a relatively small subset of items.

- **AvgPopularity@10 ≈ 1333**  
  The high average popularity of recommended items suggests a **strong popularity bias**.  
  The model heavily favors popular items, which may improve hit rate slightly but reduces personalization and novelty.

---

#### User Base Context

- **Evaluated on 5,553 users**, which provides statistically meaningful results.  
  The observed weaknesses are therefore **systematic rather than anecdotal**.

## BPRMF (Bayesian Personalized Ranking Matrix Factorization)

**BPRMF** is a recommendation model designed specifically for **implicit feedback** scenarios, where we observe interactions (views, clicks, purchases) rather than explicit ratings.

It combines:
- **Matrix Factorization (MF)** for latent user–item representations  
- **Bayesian Personalized Ranking (BPR)** as a pairwise ranking loss

---

### Motivation

In many real-world recommender systems:
- We do **not** know what users dislike
- We only observe **positive interactions**
- Missing interactions are ambiguous (unknown, not negative)

Traditional rating-based losses (MSE, RMSE) are poorly suited for this setting.

**BPRMF solves this by learning to rank observed items higher than unobserved ones.**

---

### Core Idea

For a given user **u**:
- An interacted item **i** should be ranked **higher** than a non-interacted item **j**

Formally, the model learns from triplets:

$$
(u, i, j)
$$

where:
- $\large i$ is a positive (observed) item
- $\large j$ is a negative (sampled) item

---

### Model Structure (Matrix Factorization)

Each user and item is represented by a latent vector:

- User embedding: $\large p_u \in R^k$
- Item embedding: $\large q_i \in R^k$

The predicted preference score is:

$$
\huge \hat{s}(u, i) = p_u · q_i
$$

---

### BPR Loss Function

The optimization objective is:

$$
\huge L = - \sum{}{\log \sigma( \hat{s}(u, i) - \hat{s}(u, j)) + \lambda ||\theta||^2} 
$$

Where:
- $\large \sigma$ is the sigmoid function
- $\large (u, i, j)$ is a sampled triplet
- $\large \theta$ are model parameters
- $\large \lambda$ is the regularization strength

This encourages:

$$
\huge \hat{s}(u, i) > \hat{s}(u, j)
$$

### Training Process

1. Sample a user $\large u$
2. Sample a positive item $\large i$ (from interactions)
3. Sample a negative item $\large j$ (not interacted)
4. Update embeddings via stochastic gradient descent

This process is repeated over many epochs.

---

### Why Use BPRMF?

**Advantages**
- Designed for implicit feedback
- Directly optimizes ranking quality
- Simple and efficient
- Strong baseline for recommender systems

**Limitations**
- Requires negative sampling
- No side features (pure collaborative filtering)
- Performance depends on sampling strategy

### Dataset

In [24]:
class TripletDatasetPopularHard(torch.utils.data.Dataset):
    def __init__(self, train_pos, X_ui, top_pop_iidx, n_triplets, seed=42):
        rng = np.random.default_rng(seed)
        pos = train_pos[["u_idx","i_idx"]].to_numpy()
        idx = rng.integers(0, len(pos), size=n_triplets)

        u = pos[idx,0].astype(np.int64)
        i_pos = pos[idx,1].astype(np.int64)
        i_neg = np.empty(n_triplets, dtype=np.int64)

        for t in range(n_triplets):
            liked = set(X_ui.getrow(u[t]).indices)
            j = int(top_pop_iidx[int(rng.integers(0, len(top_pop_iidx)))])
            while j in liked:
                j = int(top_pop_iidx[int(rng.integers(0, len(top_pop_iidx)))])
            i_neg[t] = j

        self.u = torch.tensor(u, dtype=torch.long)
        self.i_pos = torch.tensor(i_pos, dtype=torch.long)
        self.i_neg = torch.tensor(i_neg, dtype=torch.long)

    def __len__(self): return len(self.u)
    def __getitem__(self, idx): return self.u[idx], self.i_pos[idx], self.i_neg[idx]

### Model

In [25]:
class BPRMF(torch.nn.Module):
    def __init__(self, n_users, n_items, emb_dim=64):
        super().__init__()
        self.user_emb = torch.nn.Embedding(n_users, emb_dim)
        self.item_emb = torch.nn.Embedding(n_items, emb_dim)
        torch.nn.init.normal_(self.user_emb.weight, std=0.01)
        torch.nn.init.normal_(self.item_emb.weight, std=0.01)

    def forward(self, u, i):
        return (self.user_emb(u) * self.item_emb(i)).sum(dim=1)

### Train function

In [26]:
top_pop_movie_ids = item_popularity.index[:2000].tolist()
top_pop_iidx = np.array([item2idx[m] for m in top_pop_movie_ids if m in item2idx], dtype=np.int64)

def sample_hard_neg_popular(u_idx: int, X_ui, top_pop_iidx, rng):
    liked = set(X_ui.getrow(u_idx).indices)
    j = int(top_pop_iidx[int(rng.integers(0, len(top_pop_iidx)))])
    while j in liked:
        j = int(top_pop_iidx[int(rng.integers(0, len(top_pop_iidx)))])
    return j

In [27]:
import torch
from torch.utils.data import DataLoader

def train_neural_mf_bpr(
    model,
    train_pos,
    X_ui,
    n_items,
    test_df,
    epochs=10,
    batch_size=4096,
    lr=1e-3,
    weight_decay=1e-6,
    triplets_per_epoch=500_000,   # начни меньше, 2M может быть тяжело
    device="cuda" if torch.cuda.is_available() else "cpu",
):
    model = model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best = -1
    best_state = None
    patience = 3
    bad = 0

    for epoch in range(1, epochs + 1):
        ds = TripletDatasetPopularHard(
            train_pos=train_pos,
            X_ui=X_ui,
            top_pop_iidx=top_pop_iidx,
            n_triplets=triplets_per_epoch,
            seed=2000 + epoch,
        )
        dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)

        model.train()
        total_loss = 0.0
        for u, ip, ineg in dl:
            u, ip, ineg = u.to(device), ip.to(device), ineg.to(device)
            opt.zero_grad()

            s_pos = model(u, ip)
            s_neg = model(u, ineg)

            loss = -torch.nn.functional.logsigmoid(s_pos - s_neg).mean()
            loss.backward()
            opt.step()
            total_loss += loss.item() * len(u)

        print(f"Epoch {epoch}/{epochs} - bpr_loss: {total_loss/len(ds):.4f}")

        if epoch % 2 == 0:
            r, n = evaluate_model(
                recommend_fn=lambda m, uid, k=10: recommend_nmf(m, uid, k=k, device=device),
                model=model,
                test_df=test_df,
                k=10,
            )
            print(f"  Eval Recall@10={r:.4f} NDCG@10={n:.4f}")
            if r > best:
                best = r
                best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
                bad = 0
            else:
                bad += 1
            if bad >= patience:
                break

    model.load_state_dict(best_state)

    return model

### Train model

In [28]:
n_users, n_items = X_ui.shape
nmf_bpr = BPRMF(n_users=n_users, n_items=n_items, emb_dim=64)

nmf_bpr = train_neural_mf_bpr(
    model=nmf_bpr,
    train_pos=train_pos,
    X_ui=X_ui,
    n_items=n_items,
    test_df=test_df,
    epochs=20,
    batch_size=4096,
    lr=1e-3,
    triplets_per_epoch=500_000,
)

Epoch 1/20 - bpr_loss: 0.6835
Epoch 2/20 - bpr_loss: 0.5152
  Eval Recall@10=0.0431 NDCG@10=0.0307
Epoch 3/20 - bpr_loss: 0.4244
Epoch 4/20 - bpr_loss: 0.3967
  Eval Recall@10=0.0475 NDCG@10=0.0336
Epoch 5/20 - bpr_loss: 0.3747
Epoch 6/20 - bpr_loss: 0.3591
  Eval Recall@10=0.0512 NDCG@10=0.0362
Epoch 7/20 - bpr_loss: 0.3447
Epoch 8/20 - bpr_loss: 0.3318
  Eval Recall@10=0.0492 NDCG@10=0.0353
Epoch 9/20 - bpr_loss: 0.3196
Epoch 10/20 - bpr_loss: 0.3101
  Eval Recall@10=0.0450 NDCG@10=0.0335
Epoch 11/20 - bpr_loss: 0.2996
Epoch 12/20 - bpr_loss: 0.2911
  Eval Recall@10=0.0417 NDCG@10=0.0319


### Validate

In [29]:
recall10_bpr, ndcg10_bpr = evaluate_model(
    recommend_fn=lambda m, uid, k=10: recommend_nmf(m, uid, k=k),
    model=nmf_bpr,
    test_df=test_df,
    k=10,
)
recall10_bpr, ndcg10_bpr

(0.05118854673149649, 0.03621345174996731)

### Metrics

In [30]:
recommend_nmf_fn = lambda uid, k: recommend_nmf(nmf_bpr, uid, k=k)
metrics_nmf = evaluate_all(
    recommend_nmf_fn,
    test_df,
    users_for_coverage=test_df["user_id"].unique()[:1000],
    all_items=movies_df["movie_id"].unique(),
    item_popularity=item_popularity.to_dict(),
    k=10,
)
metrics_nmf

{'recall@10': 0.05118854673149649,
 'ndcg@10': 0.03621345174996731,
 'mrr@10': 0.05245262820897612,
 'hitrate@10': 0.14856834143706105,
 'n_users_eval': 5553,
 'coverage@10': 0.04300798351789853,
 'avg_popularity@10': 1936.5911}

### Observations

Based on the evaluation metrics for the **BPRMF (Bayesian Personalized Ranking Matrix Factorization)** model, the following conclusions can be drawn.

---

#### Overall Recommendation Quality

- **Recall@10 ≈ 0.050**  
  The model retrieves at least one relevant item for about **5.0% of users** within the top-10 recommendations.  
  This is **lower than the neural model**, indicating weaker ability to recover relevant items.

- **NDCG@10 ≈ 0.036**  
  The low NDCG score suggests that **relevant items are poorly ranked**, even when they are present in the recommendation list.

- **MRR@10 ≈ 0.052**  
  The first relevant item tends to appear **late in the ranking**, confirming limited ranking effectiveness.

---

#### Hit-Based Evaluation

- **HitRate@10 ≈ 0.145**  
  Only **14.5% of users** receive at least one relevant recommendation in the top-10.  
  This reflects **unstable performance across users** and weaker personalization.

---

#### Coverage and Popularity Bias

- **Coverage@10 ≈ 0.042**  
  Only **4.2% of the item catalog** appears in recommendations, which is **extremely low**.  
  The model strongly concentrates on a very small subset of items.

- **AvgPopularity@10 ≈ 1923**  
  The very high average popularity indicates a **severe popularity bias**.  
  The model overwhelmingly recommends globally popular items, limiting novelty and personalization.

---

#### User Base Context

- **Evaluated on 5,553 users**, making the results statistically reliable.  
  The observed issues are consistent across the evaluation population.

## Model Comparison

Based on the provided evaluation metrics, the **neural network–based recommender** clearly outperforms the **BPRMF (Bayesian Personalized Ranking Matrix Factorization)** model across all key dimensions.

---

## 1. Which Model Performs Better?

### Summary Table (Qualitative)

| Metric                | Neural Model | BPRMF | Better |
|-----------------------|--------------|-------|--------|
| Recall@10             | Higher       | Lower | Neural |
| NDCG@10               | Higher       | Lower | Neural |
| MRR@10                | Higher       | Lower | Neural |
| HitRate@10            | Higher       | Lower | Neural |
| Item Coverage@10      | Much higher  | Very low | Neural |
| Avg Popularity@10     | Lower        | Much higher | Neural |

**Conclusion:**  
➡️ The **neural recommender model performs better overall** in terms of relevance, ranking quality, personalization, and diversity.

---

## 2. Why Does BPRMF Perform Worse?

Several structural and data-related reasons explain the weaker performance of BPRMF in this project.

---

### 2.1 Model Capacity and Expressiveness

- **BPRMF is a linear latent-factor model**
  - Captures only user–item interactions via dot products
  - Cannot model complex, non-linear user preferences

- **Neural models**
  - Learn non-linear representations
  - Better capture higher-order interaction patterns
  - Can implicitly model user behavior heterogeneity

➡️ This limits BPRMF’s ability to rank relevant items accurately.

---

### 2.2 Popularity Bias

- **AvgPopularity@10 ≈ 1923 (BPRMF)** vs much lower in the neural model
- **Coverage@10 ≈ 4.2%**, indicating extreme item concentration

BPRMF tends to:
- Overfit to globally popular items
- Under-represent niche or long-tail items

➡️ This leads to poor personalization and low catalog coverage.

---

### 2.3 Optimization Objective Mismatch

- BPRMF optimizes a **pairwise ranking loss**
  - Sensitive to negative sampling strategy
  - May converge to shallow popularity-based solutions

- Neural models often:
  - Use richer training signals
  - Benefit from better regularization and capacity

➡️ The optimization may not align well with top-K ranking quality in this dataset.

---

### 2.4 Data Characteristics

The dataset likely exhibits:
- Sparse user–item interactions
- Long-tail item distribution
- High user heterogeneity

These conditions:
- Hurt shallow latent-factor models like BPRMF
- Favor higher-capacity neural architectures

---

### 2.5 Training and Tuning Sensitivity

BPRMF performance is highly sensitive to:
- Latent dimension size
- Learning rate and regularization
- Number of negative samples
- Number of training epochs

Without extensive tuning, it often underperforms modern neural approaches.

---

## 3. Final Conclusion

- ✅ **Neural model**  
  - Better relevance and ranking
  - Higher personalization
  - Broader item coverage
  - Lower popularity bias

- ❌ **BPRMF**
  - Lower recommendation quality
  - Severe popularity bias
  - Very limited diversity

## Save model

In [31]:
nmf.eval()
u = torch.tensor([0, 1, 2, 3], dtype=torch.long)
i = torch.tensor([10, 11, 12, 13], dtype=torch.long)
torch.onnx.export(
    nmf,
    (u, i),
    os.path.join(MODELS, "pytorch_model.pt"),
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    input_names=["user_id", "item_id"],
    output_names=["logit"],
    dynamic_axes={
        "user_id": {0: "batch"},
        "item_id": {0: "batch"},
        "logit":   {0: "batch"},
    },
)

/var/folders/v0/8r5h4ym13fb_mzmqc1hfkt580000gn/T/ipykernel_26225/2508353520.py:4: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0121 22:33:41.939000 26225 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0121 22:33:42.312000 26225 torch/onnx/_internal/exporter/_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `NeuralMF([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NeuralMF([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 17},
            producer_name='pytorch',
            producer_version='2.9.1',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"user_id"<INT64,[s44]>,
                %"item_id"<INT64,[s44]>
            ),
            outputs=(
                %"logit"<FLOAT,[s44]>
            ),
            initializers=(
                %"mlp.0.bias"<FLOAT,[128]>{TorchTensor(...)},
                %"mlp.3.weight"<FLOAT,[1,128]>{TorchTensor(...)},
                %"mlp.3.bias"<FLOAT,[1]>{TorchTensor<FLOAT,[1]>(Parameter containing: tensor([0.0320], requires_grad=True), name='mlp.3.bias')},
                %"user_emb.weight"<FLOAT,[6038,64]>{TorchTensor(...)},
                %"item_emb.weight"<FLOAT,[3525,64]>{TorchTensor(...)},
                %"mlp.0.weight"<FLOAT,[128,128]>{TorchTensor(...)},
               